# Bibliotecas

In [1]:
import json
import urllib
import requests
import pandas as pd
import numpy as np
pd.set_option('max_columns', None)
pd.set_option('float_format', '{:f}'.format)

# Ingestão de dados

In [2]:
minha_lista    = pd.read_csv('../data/raw/minha_lista.csv',index_col=0,low_memory=False).reset_index(drop=True)
ja_encontrados = pd.read_csv('../data/raw/cep_ibge.csv',low_memory=False).reset_index(drop=True)
nao_encontrado = pd.read_csv('../data/raw/nao_encontrado.csv',index_col=0,low_memory=False,names=['cep']).reset_index(drop=True)

# Ajuste de CEP
As colunas CEP tem característica de `INT`. Vamos transformar em string e completar com zeros

In [3]:
minha_lista['cep'] = minha_lista['cep'].map(int).apply(lambda x: '{0:0>8}'.format(x))
ja_encontrados['cep'] = ja_encontrados['cep'].map(int).apply(lambda x: '{0:0>8}'.format(x))
nao_encontrado['cep'] = nao_encontrado['cep'].map(int).apply(lambda x: '{0:0>8}'.format(x))

# Removendo valores em comum

## já encontrados
Se na tabela `ja_encontrados` já conter o cep que buscamos, não há porque procurar pois já temos a informação.

In [4]:
minha_lista = minha_lista[~minha_lista['cep'].isin(ja_encontrados['cep'])]

## não encontrados
Se na tabela `nao_encontrado` já conter o cep que buscamos, não há porque procurar pois não como encontrar ela.

In [5]:
# APAGAR
# nao_encontrado = nao_encontrado.head(1)

In [6]:
minha_lista = minha_lista[~minha_lista['cep'].isin(nao_encontrado['cep'])]

#  Ajustes para rodar

In [7]:
# Vamos remover tudo que é NAN
minha_lista = minha_lista.dropna()

In [8]:
# Se nessa lista contiver caracteres especiais, vamos ajustá-los
minha_lista['cep'] =  minha_lista['cep'].str.replace("\.|\-|\,","")

In [9]:
# Vamos tagear tudo aquilo que não for exclusivamente numérico
minha_lista['flg_cep'] = minha_lista['cep'].astype(str).str.isnumeric()

In [10]:
# Vamos manter os registros que são exclusivamente numéricos
CEPs = minha_lista[minha_lista['flg_cep']==True].drop(['flg_cep'],1)

In [11]:
# Se houver registros duplicados, vamos aqui removê-los
CEPs = CEPs.drop_duplicates()

In [12]:
# Vamos transformar esse resultado em lista
lista_ceps=list(CEPs['cep'])

# Reconhecendo CEP
Vamos rodar duas apis. Vamos dar prioridade à awesomeapi que é mais completa e em caso negativo, vamos buscar na postmon.

In [13]:
lista_ceps=list(CEPs['cep'])

In [14]:
# Vamos criar um array pois dessa forma podemos salvar resultados intermediários.
meu_cep = np.array_split(lista_ceps, int(len(lista_ceps)/round(len(lista_ceps)/50)))
len(meu_cep)

45

In [ ]:
# A função abaixo deve buscar na api um cep e retornar o código IBGE daquele valor
for i, cepzin in enumerate(meu_cep):

    # lista para salvar dados de ibge/cep
    awesomeapi=[]
    postmon=[]

    # lista para armazenar ceps não encontrados
    error = []
    
    # comment
    for cep in (cepzin):
        print(cep, end=" - ")
        try:
            resp = urllib.request.urlopen('https://cep.awesomeapi.com.br/json/'+cep)
            awesomeapi.append(json.loads(resp.read()))
            print("1")
        except:
            try:
                url = ('https://api.postmon.com.br/v1/cep/'+cep)
                postmon.append(requests.get(url, timeout=0.1).json())
                print("2")
            except:
                error.append(cep)
                print("3")
                
    pd.DataFrame(awesomeapi).to_csv('../data/processed/temp/partial_awesomeapi_'+str(i)+'.csv')
    pd.DataFrame(postmon).to_csv('../data/processed/temp/partial_postmon_'+str(i)+'.csv')
    pd.DataFrame(error).to_csv('../data/processed/temp/partial_error_'+str(i)+'.csv')

# Leitura dos resultados

In [ ]:
# junção de agregação dos resultados parciais
from pathlib import Path
def agregador(file=None, path='../data/processed/temp/'):
    data_dir = Path(path)
    df = pd.concat(
        pd.read_csv(csv_file, index_col=0)
        for csv_file in data_dir.glob(file+'*'))
    return df

In [ ]:
awesomeapi = agregador(file='partial_awesomeapi_')
awesomeapi = awesomeapi.drop_duplicates()

In [ ]:
postmon = agregador(file='partial_postmon_')
postmon = postmon.drop_duplicates()

In [ ]:
error = agregador(file='partial_error_')
error = error.drop_duplicates()

# Salvando backup

In [ ]:
from shutil import copyfile
from datetime import datetime
dt_string = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

try:
    copyfile('../data/raw/cep_ibge.csv', '../data/processed/cep_ibge_' + dt_string + '.csv')
    copyfile('../data/raw/minha_lista.csv', '../data/processed/minha_lista' + dt_string + '.csv')
    copyfile('../data/raw/nao_encontrado.csv', '../data/processed/nao_encontrado' + dt_string + '.csv')

except:
    print('erro')
    pass

# Salvando resultados

In [ ]:
awesomeapi.to_csv('../data/processed/awesomeapi.csv')
postmon.to_csv('../data/processed/postmon.csv')
error.to_csv('../data/processed/nao_encontrado.csv')

## postmon


In [ ]:
postmon['estado_info'] = postmon['estado_info'].fillna("{}")
postmon['cidade_info'] = postmon['cidade_info'].fillna("{}")

In [ ]:
import ast
postmon["cidade_info"] =  postmon["cidade_info"].map(lambda d : ast.literal_eval(d))

In [ ]:
postmon = pd.concat([postmon.drop(['cidade_info'], axis=1), postmon['cidade_info'].apply(pd.Series)], axis=1)

In [ ]:
try:
    postmon = postmon[['bairro', 'cidade', 'cep', 'estado','codigo_ibge', 'logradouro']]
except:
    postmon['logradouro']=np.nan
    postmon = postmon[['bairro', 'cidade', 'cep', 'estado','codigo_ibge', 'logradouro']]
postmon

In [ ]:
postmon = postmon.rename(columns={
     'bairro':'district',
     'cidade':'city',
     'estado':'state',
     'logradouro':'address',
     'codigo_ibge':'city_ibge',
})
postmon

## resultado final

In [ ]:
ja_encontrados = pd.read_csv('../data/raw/cep_ibge.csv',index_col=0,low_memory=False)

In [ ]:
resultado_final = pd.concat([ja_encontrados,awesomeapi],0)
resultado_final.head()

In [ ]:
resultado_final = pd.concat([resultado_final,postmon],0).reset_index(drop=True)
resultado_final['cep'] = resultado_final['cep'].map(int).apply(lambda x: '{0:0>8}'.format(x))
resultado_final.tail()

# Salvar

In [ ]:
resultado_final.to_csv('../data/raw/cep_ibge.csv')

In [ ]:
error.to_csv('../data/raw/nao_encontrado.csv')